In [1]:
# -*- encoding:utf-8 -*-
#-*- coding: cp950 -*-

import os
import sys

import pyspark
import random

import matplotlib.pyplot as plt
import numpy as np
import numpy
from matplotlib import cm
import pandas as pd
from numpy import concatenate
from pandas import concat

from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import col  
import pyspark.sql.types
from pyspark.sql.types import DoubleType
from pyspark.sql import Row
from pyspark.sql.types import Row
from pyspark.sql.functions import udf

from pyspark import since, SparkContext, keyword_only

from pyspark.ml.common import _java2py, _py2java
from pyspark.ml.wrapper import _jvm
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaEstimator, JavaModel
from pyspark.ml.param.shared import *
from pyspark.ml.common import inherit_doc

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import QuantileDiscretizer, VectorSlicer
from pyspark.ml.feature import ChiSqSelector

from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.feature import Binarizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.stat import Summarizer

from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import PCA
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.feature import DCT
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import OneHotEncoder, Normalizer, StandardScaler
from pyspark.ml.feature import MinMaxScaler, MaxAbsScaler, Bucketizer
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.feature import SQLTransformer

from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan

from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import LDA
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import PowerIterationClustering


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LogisticRegression , OneVsRest
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import FMClassifier

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import FMRegressor

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.fpm import FPGrowth
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD


In [2]:
#數據集:  covtype森林覆蓋
#無欄位名稱的橫列
#第1欄位:編碼, 有12個屬性
#第2至11欄位是數值型, 
#其他是二元類別型
#最後一個是Y結果變數Cover_Type,有7個屬性值 integer: 1 to 7

In [3]:
sc = SparkContext('local')
spark = SparkSession(sc)

21/07/26 07:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:

dtt= spark.read.format('csv').option("header",'false').load("/home/jovyan/work/pySpark/pySpark_james/data/covtype.csv")
print('data:',dtt.count() )
dtt.show(3)

21/07/26 07:30:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


data: 581012
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0|_c1|_c2|_c3|_c4| _c5|_c6|_c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|
+----+---+---+---+---+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|2596| 51|  3|258|  0| 510|221|232|148|6279|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   

In [5]:
dtt.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [6]:
featuresCols=dtt.columns[:54]
df= dtt.select([ col(column).cast("double").alias(column) 
    for column in dtt.columns])
df=df.withColumn("label", df["_c54"] -1).drop("_c54") 
train_df, test_df = df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="features")
mod= RandomForestClassifier(labelCol="label",featuresCol='features')#,maxDepth =5,maxBins=20)
PL= Pipeline(stages=[vectorAssembler,mod])
PLmod=PL.fit(train_df)
pred= PLmod.transform(test_df)
evaluator = MulticlassClassificationEvaluator(
                            labelCol="label", predictionCol="prediction", 
                            metricName="accuracy")
pred=PLmod.transform(test_df)
res=pred.withColumnRenamed("_c0", "海拔") \
    .withColumnRenamed("_c1", "方位") \
     .withColumnRenamed("_c2", "斜率") \
    .withColumnRenamed("_c3", "垂直距離") \
   .withColumnRenamed("_c4", "水平距離") \
   .withColumnRenamed("_c5", "陰影")           
res.select("海拔","方位","斜率","垂直距離" , "水平距離","陰影","label","prediction").show(10)
accuracy = evaluator.evaluate(pred)
print('Accuracy=',accuracy)

21/07/26 07:31:18 WARN MemoryStore: Not enough space to cache rdd_24_0 in memory! (computed 132.2 MiB so far)
21/07/26 07:31:18 WARN BlockManager: Persisting block rdd_24_0 to disk instead.


+------+----+----+--------+--------+-----+-----+----------+
|  海拔|方位|斜率|垂直距離|水平距離| 陰影|label|prediction|
+------+----+----+--------+--------+-----+-----+----------+
|1859.0|18.0|12.0|    67.0|    11.0| 90.0|  2.0|       2.0|
|1872.0|35.0|21.0|   120.0|    18.0| 85.0|  5.0|       2.0|
|1877.0|19.0|18.0|    85.0|    25.0|108.0|  2.0|       2.0|
|1877.0|28.0|22.0|   127.0|    35.0| 85.0|  5.0|       2.0|
|1880.0|34.0|20.0|   124.0|    17.0| 90.0|  5.0|       2.0|
|1883.0|27.0|24.0|   120.0|    24.0|108.0|  5.0|       2.0|
|1885.0|30.0|19.0|   134.0|    35.0|162.0|  2.0|       2.0|
|1888.0| 9.0|25.0|   120.0|    32.0| 90.0|  5.0|       2.0|
|1888.0|17.0|19.0|   127.0|    36.0|150.0|  2.0|       2.0|
|1888.0|33.0|22.0|   150.0|    46.0|108.0|  5.0|       2.0|
+------+----+----+--------+--------+-----+-----+----------+
only showing top 10 rows



Accuracy= 0.668781081034788
